# GoogLeNet - Inception 모듈 구현

## Naive 버전

In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D
from tensorflow.keras.layers import MaxPooling2D, concatenate

model_input = Input(shape=(28,28,3))
pre = Dense(192)(model_input)
conv1 = Conv2D(64, 1, padding='same')(pre)
conv2 = Conv2D(128, 3, padding='same')(pre)
conv3 = Conv2D(32, 5, padding='same')(pre)
pool = MaxPooling2D(pool_size=(3,3), strides=1, padding='same')(pre)
model_output = concatenate([ conv1, conv2, conv3, pool])

model = Model(inputs=model_input, outputs=model_output)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 28, 28, 3)]          0         []                            
                                                                                                  
 dense_2 (Dense)             (None, 28, 28, 192)          768       ['input_3[0][0]']             
                                                                                                  
 conv2d_6 (Conv2D)           (None, 28, 28, 64)           12352     ['dense_2[0][0]']             
                                                                                                  
 conv2d_7 (Conv2D)           (None, 28, 28, 128)          221312    ['dense_2[0][0]']             
                                                                                            

## 1x1 Conv층을 추가한 Bottle Neck 구조

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D
from tensorflow.keras.layers import MaxPooling2D, concatenate

model_input = Input(shape=(28,28,3))
pre = Dense(192)(model_input)

conv1 = Conv2D(64, 1, padding='same')(pre)

conv1_2 = Conv2D(96, 1, padding='same')(pre)
conv2 = Conv2D(128, 3, padding='same')(conv1_2)

conv1_3 = Conv2D(16, 1, padding='same')(pre)
conv3 = Conv2D(32, 5, padding='same')(conv1_3)

pool = MaxPooling2D(pool_size=(3,3), strides=1, padding='same')(pre)
conv1_4 = Conv2D(32, 1, padding='same')(pool)
model_output = concatenate([ conv1, conv2, conv3, conv1_4])

model = Model(inputs=model_input, outputs=model_output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 28, 28, 3)]          0         []                            
                                                                                                  
 dense (Dense)               (None, 28, 28, 192)          768       ['input_1[0][0]']             
                                                                                                  
 conv2d_1 (Conv2D)           (None, 28, 28, 96)           18528     ['dense[0][0]']               
                                                                                                  
 conv2d_3 (Conv2D)           (None, 28, 28, 16)           3088      ['dense[0][0]']               
                                                                                              